## [1] 유튜브 영상 크롤링

In [14]:
!pip install selenium
!pip install webdriver-manager

In [15]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver

pip install selenium

UsageError: Cell magic `%%shell` not found.


In [16]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

'/Users/yewonsong/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/120/chromedriver'

In [17]:
driver = webdriver.Chrome(options=chrome_options)

In [18]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

import time
import pandas as pd
import re

In [19]:
!pip install pytube
from pytube import YouTube

### MBC 2015~2022

In [20]:
import random

In [22]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

In [23]:
#예시 after:2016-12-31 before:2018-1-1 음주운전
yr_interest = [2015,2016,2017,2018,2019,2020,2021,2022]
for interest in yr_interest:
    bf = interest-1
    aft = interest+1
    
    yt_url = f'https://www.youtube.com/@MBCNEWS11/search?query=after%3A{bf}-12-31%20before%3A{aft}-1-1%20%EC%9D%8C%EC%A3%BC%EC%9A%B4%EC%A0%84'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(yt_url)
    
    # 브라우저 로드가 완료되기 위한 시간
    time.sleep(2)
    body = driver.find_element(By.TAG_NAME, value='body')

    scroll()

    # selenium을 이용해서 HTML문서를 변환한 후에는 반드시 브라우저를 종료
    html = bs(driver.page_source, 'lxml')
    driver.close()

    titleList = []
    contentUrlList = []
    viewcountList = []
    commentcountList = []

    for content in html.select('a#video-title'):
        #제목 수집
        title = content.get('title')

        #URL 수집
        content_url = 'https://www.youtube.com'+content.get('href')

        #조회수 수집
        label = content.get('aria-label')
        split_label = label.split()
        idx = label.split().index('조회수')
        viewcount = split_label[idx+1]
        viewcount=viewcount.replace(",","")
        viewcount=viewcount.replace("회","")

        #빈 리스트에 추가
        titleList.append(title)
        contentUrlList.append(content_url)
        viewcountList.append(int(viewcount))

    yt_dic = {'영상제목' : titleList,'영상주소' : contentUrlList,'조회수' : viewcountList}

    # 데이터 프레임 생성
    yt_df = pd.DataFrame(yt_dic)

    #날짜 가져오기
    dateList = []
    for link in yt_df['영상주소']:
        yt = YouTube(link)
        dateList.append(yt.publish_date.date())

    yt_df['날짜'] = dateList

    #엑셀 파일로 저장
    yt_df.to_excel(f'MBC_{interest}.xlsx')

스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료


### YTN 2015~2022

In [24]:
yr_interest = [2015,2016,2017,2018,2019,2020,2021,2022]
for interest in yr_interest:
    bf = interest-1
    aft = interest+1
    
    yt_url = f'https://www.youtube.com/@ytnnews24/search?query=after%3A{bf}-12-31%20before%3A{aft}-1-1%20%EC%9D%8C%EC%A3%BC%EC%9A%B4%EC%A0%84'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(yt_url)
    
    # 브라우저 로드가 완료되기 위한 시간
    time.sleep(2)
    body = driver.find_element(By.TAG_NAME, value='body')

    scroll()

    # selenium을 이용해서 HTML문서를 변환한 후에는 반드시 브라우저를 종료
    html = bs(driver.page_source, 'lxml')
    driver.close()

    titleList = []
    contentUrlList = []
    viewcountList = []
    commentcountList = []

    for content in html.select('a#video-title'):
        #제목 수집
        title = content.get('title')

        #URL 수집
        content_url = 'https://www.youtube.com'+content.get('href')

        #조회수 수집
        label = content.get('aria-label')
        split_label = label.split()
        idx = label.split().index('조회수')
        viewcount = split_label[idx+1]
        viewcount=viewcount.replace(",","")
        viewcount=viewcount.replace("회","")

        #빈 리스트에 추가
        titleList.append(title)
        contentUrlList.append(content_url)
        viewcountList.append(int(viewcount))

    yt_dic = {'영상제목' : titleList,'영상주소' : contentUrlList,'조회수' : viewcountList}

    # 데이터 프레임 생성
    yt_df = pd.DataFrame(yt_dic)

    #날짜 가져오기
    dateList = []
    for link in yt_df['영상주소']:
        yt = YouTube(link)
        dateList.append(yt.publish_date.date())

    yt_df['날짜'] = dateList

    #엑셀 파일로 저장
    yt_df.to_excel(f'YTN_{interest}.xlsx')

스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료


URLError: <urlopen error [Errno 60] Operation timed out>

### SBS 2015~2022

In [26]:
yr_interest = [2015,2016,2017,2018,2019,2020,2021,2022]
for interest in yr_interest:
    bf = interest-1
    aft = interest+1
    
    yt_url = f'https://www.youtube.com/@sbsnews8/search?query=after%3A{bf}-12-31%20before%3A{aft}-1-1%20%EC%9D%8C%EC%A3%BC%EC%9A%B4%EC%A0%84'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(yt_url)
    
    # 브라우저 로드가 완료되기 위한 시간
    time.sleep(2)
    body = driver.find_element(By.TAG_NAME, value='body')

    scroll()

    # selenium을 이용해서 HTML문서를 변환한 후에는 반드시 브라우저를 종료
    html = bs(driver.page_source, 'lxml')
    driver.close()

    titleList = []
    contentUrlList = []
    viewcountList = []
    commentcountList = []

    for content in html.select('a#video-title'):
        #제목 수집
        title = content.get('title')

        #URL 수집
        content_url = 'https://www.youtube.com'+content.get('href')

        #조회수 수집
        label = content.get('aria-label')
        split_label = label.split()
        idx = label.split().index('조회수')
        viewcount = split_label[idx+1]
        viewcount=viewcount.replace(",","")
        viewcount=viewcount.replace("회","")

        #빈 리스트에 추가
        titleList.append(title)
        contentUrlList.append(content_url)
        viewcountList.append(int(viewcount))

    yt_dic = {'영상제목' : titleList,'영상주소' : contentUrlList,'조회수' : viewcountList}

    # 데이터 프레임 생성
    yt_df = pd.DataFrame(yt_dic)

    #날짜 가져오기
    dateList = []
    for link in yt_df['영상주소']:
        yt = YouTube(link)
        dateList.append(yt.publish_date.date())

    yt_df['날짜'] = dateList

    #엑셀 파일로 저장
    yt_df.to_excel(f'SBS_{interest}.xlsx')

스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료


### JTBC 2015~2022

In [27]:
yr_interest = [2015,2016,2017,2018,2019,2020,2021,2022]
for interest in yr_interest:
    bf = interest-1
    aft = interest+1
    
    yt_url = f'https://www.youtube.com/@jtbc_news/search?query=after%3A{bf}-12-31%20before%3A{aft}-1-1%20%EC%9D%8C%EC%A3%BC%EC%9A%B4%EC%A0%84'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(yt_url)
    
    # 브라우저 로드가 완료되기 위한 시간
    time.sleep(2)
    body = driver.find_element(By.TAG_NAME, value='body')

    scroll()

    # selenium을 이용해서 HTML문서를 변환한 후에는 반드시 브라우저를 종료
    html = bs(driver.page_source, 'lxml')
    driver.close()

    titleList = []
    contentUrlList = []
    viewcountList = []
    commentcountList = []

    for content in html.select('a#video-title'):
        #제목 수집
        title = content.get('title')

        #URL 수집
        content_url = 'https://www.youtube.com'+content.get('href')

        #조회수 수집
        label = content.get('aria-label')
        split_label = label.split()
        idx = label.split().index('조회수')
        viewcount = split_label[idx+1]
        viewcount=viewcount.replace(",","")
        viewcount=viewcount.replace("회","")

        #빈 리스트에 추가
        titleList.append(title)
        contentUrlList.append(content_url)
        viewcountList.append(int(viewcount))

    yt_dic = {'영상제목' : titleList,'영상주소' : contentUrlList,'조회수' : viewcountList}

    # 데이터 프레임 생성
    yt_df = pd.DataFrame(yt_dic)

    #날짜 가져오기
    dateList = []
    for link in yt_df['영상주소']:
        yt = YouTube(link)
        dateList.append(yt.publish_date.date())

    yt_df['날짜'] = dateList

    #엑셀 파일로 저장
    yt_df.to_excel(f'JTBC_{interest}.xlsx')

스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료


### KBS 2015~2022

In [28]:
yr_interest = [2015,2016,2017,2018,2019,2020,2021,2022]
for interest in yr_interest:
    bf = interest-1
    aft = interest+1
    
    yt_url = f'https://www.youtube.com/@newskbs/search?query=after%3A{bf}-12-31%20before%3A{aft}-1-1%20%EC%9D%8C%EC%A3%BC%EC%9A%B4%EC%A0%84'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(yt_url)
    
    # 브라우저 로드가 완료되기 위한 시간
    time.sleep(2)
    body = driver.find_element(By.TAG_NAME, value='body')

    scroll()

    # selenium을 이용해서 HTML문서를 변환한 후에는 반드시 브라우저를 종료
    html = bs(driver.page_source, 'lxml')
    driver.close()

    titleList = []
    contentUrlList = []
    viewcountList = []
    commentcountList = []

    for content in html.select('a#video-title'):
        #제목 수집
        title = content.get('title')

        #URL 수집
        content_url = 'https://www.youtube.com'+content.get('href')

        #조회수 수집
        label = content.get('aria-label')
        split_label = label.split()
        idx = label.split().index('조회수')
        viewcount = split_label[idx+1]
        viewcount=viewcount.replace(",","")
        viewcount=viewcount.replace("회","")

        #빈 리스트에 추가
        titleList.append(title)
        contentUrlList.append(content_url)
        viewcountList.append(int(viewcount))

    yt_dic = {'영상제목' : titleList,'영상주소' : contentUrlList,'조회수' : viewcountList}

    # 데이터 프레임 생성
    yt_df = pd.DataFrame(yt_dic)

    #날짜 가져오기
    dateList = []
    for link in yt_df['영상주소']:
        yt = YouTube(link)
        dateList.append(yt.publish_date.date())

    yt_df['날짜'] = dateList

    #엑셀 파일로 저장
    yt_df.to_excel(f'KBS_{interest}.xlsx')

스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료
스크롤 완료


## [2] 분석

In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import probplot
from statsmodels.formula.api import ols
import statsmodels.api as sm
import re
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

### JTBC, KBS, MBC, SBS, YTN의 음주운전 관련 영상 (2015~2022) 데이터 불러오기

In [ ]:
df_JTBC={}
df_KBS={}
df_MBC={}
df_SBS={}
df_YTN={}
df=[df_JTBC,df_KBS,df_MBC,df_SBS,df_YTN]

compList = ['JTBC','KBS','MBC','SBS','YTN']
yearList = [2015,2016,2017,2018,2019,2020,2021,2022]


for year in yearList:
    for i,comp in enumerate(compList):
        df_data = df[i]
        dataset_path = f"/Users/yewonsong/Desktop/EDA/{comp}/"
        file_path = os.path.join(dataset_path,f"{comp}_{year}.xlsx")
        df_data[year]=pd.read_excel(file_path)
        
        df_data[year].drop('Unnamed: 0',axis=1,inplace=True) #필요없는 컬럼 제거 
        df_data[year] = df_data[year].drop_duplicates() #중복 행 제거
        df_data[year] = df_data[year].sort_values(by='날짜') #날짜 순으로 재정렬
        df_data[year]['month']=df_data[year]['날짜'].dt.strftime('%Y-%m') #month 컬럼 생성

### 회사 별로 모든 년도 합치기 합치기

In [ ]:
df_JTBC_all = df_JTBC[2015]
for year in yearList[1:]:
    df_JTBC_all = pd.concat([df_JTBC_all, df_JTBC[year]],axis=0).reset_index(drop=True)
    df_JTBC_all = df_JTBC_all.drop_duplicates()
    df_JTBC_all.reset_index(drop=True,inplace=True)

df_KBS_all = df_KBS[2015]
for year in yearList[1:]:
    df_KBS_all = pd.concat([df_KBS_all, df_KBS[year]],axis=0).reset_index(drop=True)
    df_KBS_all = df_KBS_all.drop_duplicates()
    df_KBS_all.reset_index(drop=True,inplace=True)

df_MBC_all = df_MBC[2015]
for year in yearList[1:]:
    df_MBC_all = pd.concat([df_MBC_all, df_MBC[year]],axis=0).reset_index(drop=True)
    df_MBC_all = df_MBC_all.drop_duplicates()
    df_MBC_all.reset_index(drop=True,inplace=True)

df_SBS_all = df_SBS[2015]
for year in yearList[1:]:
    df_SBS_all = pd.concat([df_SBS_all, df_SBS[year]],axis=0).reset_index(drop=True)
    df_SBS_all = df_SBS_all.drop_duplicates()
    df_SBS_all.reset_index(drop=True,inplace=True)

df_YTN_all = df_YTN[2015]
for year in yearList[1:]:
    df_YTN_all = pd.concat([df_YTN_all, df_YTN[year]],axis=0).reset_index(drop=True)
    df_YTN_all = df_YTN_all.drop_duplicates()
    df_YTN_all.reset_index(drop=True,inplace=True)

### 음주 관련 영상들만 남겨주기

In [ ]:
todrop=[]
keyword=['음주운전','음주뺑소니','윤창호','음주차량','음주사고','만취운전','음주단속']

#JTBC
for i, title in enumerate(df_JTBC_all['영상제목']):
    count = 0
    title_test = re.sub('[^A-Za-z가-힣]','',title)
    for key in keyword:
        if key in title_test:
            count =+ 1
    if count == 0:
        todrop.append(i)
        
df_JTBC_all.drop(todrop,axis=0,inplace=True)
df_JTBC_all.reset_index(drop=True,inplace=True)

#KBS
for i, title in enumerate(df_KBS_all['영상제목']):
    count = 0
    title_test = re.sub('[^A-Za-z가-힣]','',title)
    for key in keyword:
        if key in title_test:
            count =+ 1
    if count == 0:
        todrop.append(i)

df_KBS_all.drop(todrop,axis=0,inplace=True)
df_KBS_all.reset_index(drop=True,inplace=True)

#MBC
for i, title in enumerate(df_MBC_all['영상제목']):
    count = 0
    title_test = re.sub('[^A-Za-z가-힣]','',title)
    for key in keyword:
        if key in title_test:
            count =+ 1
    if count == 0:
        todrop.append(i)

df_MBC_all.drop(todrop,axis=0,inplace=True)
df_MBC_all.reset_index(drop=True,inplace=True)

#SBS
for i, title in enumerate(df_SBS_all['영상제목']):
    count = 0
    title_test = re.sub('[^A-Za-z가-힣]','',title)
    for key in keyword:
        if key in title_test:
            count =+ 1
    if count == 0:
        todrop.append(i)

df_SBS_all.drop(todrop,axis=0,inplace=True)
df_SBS_all.reset_index(drop=True,inplace=True)

#YTN
for i, title in enumerate(df_YTN_all['영상제목']):
    count = 0
    title_test = re.sub('[^A-Za-z가-힣]','',title)
    for key in keyword:
        if key in title_test:
            count =+ 1
    if count == 0:
        todrop.append(i)

df_YTN_all.drop(todrop,axis=0,inplace=True)
df_YTN_all.reset_index(drop=True,inplace=True)

### 월별 조회수 데이터 프레임 만들기

In [ ]:
df_JTBC_views = df_JTBC_all.groupby('month').sum('조회수')
df_KBS_views = df_KBS_all.groupby('month').sum('조회수')
df_MBC_views = df_MBC_all.groupby('month').sum('조회수')
df_SBS_views = df_SBS_all.groupby('month').sum('조회수')
df_YTN_views = df_YTN_all.groupby('month').sum('조회수')

#월별 영상수, 평균조회수 컬럼 생성
JTBC_count = df_JTBC_all.groupby('month').size().to_frame()
df_JTBC = pd.merge(df_JTBC_views,JTBC_count,left_index=True,right_index=True)
df_JTBC.columns = ['view','count']
df_JTBC['avg'] = df_JTBC['view']/df_JTBC['count']
df_JTBC.columns = ['view','count','avg_view']

KBS_count = df_KBS_all.groupby('month').size().to_frame()
df_KBS = pd.merge(df_KBS_views,KBS_count,left_index=True,right_index=True)
df_KBS.columns = ['view','count']
df_KBS['avg'] = df_KBS['view']/df_KBS['count']
df_KBS.columns = ['view','count','avg_view']

MBC_count = df_MBC_all.groupby('month').size().to_frame()
df_MBC = pd.merge(df_MBC_views,MBC_count,left_index=True,right_index=True)
df_MBC.columns = ['view','count']
df_MBC['avg'] = df_MBC['view']/df_MBC['count']
df_MBC.columns = ['view','count','avg_view']

SBS_count = df_SBS_all.groupby('month').size().to_frame()
df_SBS = pd.merge(df_SBS_views,SBS_count,left_index=True,right_index=True)
df_SBS.columns = ['view','count']
df_SBS['avg'] = df_SBS['view']/df_SBS['count']
df_SBS.columns = ['view','count','avg_view']

YTN_count = df_YTN_all.groupby('month').size().to_frame()
df_YTN = pd.merge(df_YTN_views,YTN_count,left_index=True,right_index=True)
df_YTN.columns = ['view','count']
df_YTN['avg'] = df_YTN['view']/df_YTN['count']
df_YTN.columns = ['view','count','avg_view']

### 5개 회사 합치기 df_YT

In [ ]:
#5개 회사 합치기
merged = pd.merge(df_JTBC,df_KBS,how='outer',left_on='month',right_on='month').sort_values('month')
merged = pd.merge(merged,df_MBC,how='outer',left_on='month',right_on='month').sort_values('month')
merged = pd.merge(merged,df_SBS,how='outer',left_on='month',right_on='month').sort_values('month')
merged = pd.merge(merged,df_YTN,how='outer',left_on='month',right_on='month').sort_values('month')

#컬럼 이름 재 설정 
merged.columns = ['JTBC_view','JTBC_count','JTBC_avg',
                  'KBS_view','KBS_count','KBS_avg',
                  'MBC_view','MBC_count','MBC_avg',
                  'SBS_view','SBS_count','SBS_avg',
                  'YTN_view','YTN_count','YTN_avg']

#NA 값 0으로 바꾸기 
merged = merged.fillna(0)
merged

#sum 컬럼 만들기
merged['sum_view']=merged['JTBC_view']+merged['KBS_view']+merged['MBC_view']+merged['SBS_view']+merged['YTN_view'] 
merged['sum_count']=merged['JTBC_count']+merged['KBS_count']+merged['MBC_count']+merged['SBS_count']+merged['YTN_count']
merged['avg_view']=merged['sum_view']/merged['sum_count']

#이름 바꾸기
df_yt = merged
df_yt

### 월별 교통사고 건수 가져오기

In [ ]:
#월별 사고 건수
dataset_path = "/Users/yewonsong/Desktop/EDA/data/일자별"
df_accident = {} #사고 (건)
df_fatal = {} #사망 (명)
df_injury = {} #부상 (명)
df_dataset = [df_accident, df_fatal, df_injury]

for year in range(2005,2023):
    for i in range(3):
        df_data=df_dataset[i] #df_accident, df_fatal, df_injury 각각 만들어주기
        file_path=os.path.join(dataset_path,f"일자별 음주운전사고 {year}.xls")
        if os.path.exists(file_path):
            try: 
                df_data[year]=pd.read_excel(file_path,header=1) #열 이름으로 사용할 행 지정
                
                row_to_drop=[j for j in range(40) if j%3!=i] #drop할 행 번호
                row_to_drop.append(i) #합계도 지워주기
                df_data[year].drop(row_to_drop,inplace=True,axis=0) #행 제거
                df_data[year].set_index(df_data[year].columns[0], inplace=True)
                
                col_to_drop=['사고년도','합계']
                df_data[year].drop(col_to_drop,inplace=True,axis=1)
                
                if df_data[year].index.isna().any().any(): #행에 하나라도 na가 있으면 
                    df_data[year].drop(np.nan,inplace=True,axis=0) #그 행 삭제...
                    
            except Exception as e:
                print(f"Error reading file for {i} year {year}: {e}")
        else:
            print(f"File not found for year: {year}")

In [ ]:
#전처리
for year in range(2005,2023):
    df_accident[year] = df_accident[year].replace('-',0)
    df_accident[year].astype(int)
    df_accident[year]['sum_acc']=df_accident[year].loc[:,:].sum(axis=1) #월별 사고 건수 합 컬럼 생성
    
    #날짜 열 생성
    start_date=f'{year}0101'
    end_date=f'{year}1231'
    date_list=pd.date_range(start=start_date, end=end_date, freq='M')
    dateList = []
    for date in date_list:
        date = date.strftime('%Y-%m')
        dateList.append(date)
    df_accident[year]['date']= dateList

In [ ]:
#모든 월 합쳐주기
df_acc = df_accident[2015]
for year in range(2016,2023):
    df_acc = pd.concat([df_acc,df_accident[year]],axis=0)
df_acc.set_index('date',drop=True,inplace=True)
df_acc

### 시각화

In [ ]:
#사고 추세 시각화
plt.figure(figsize=(10,5))
dates = pd.date_range(start='2015-01-01', periods=len(df_acc), freq='M')
plt.plot(dates,df_acc['sum_acc'])
plt.xlabel('Month')
plt.ylabel('Number of Accidents')
plt.title('Number of accidents by month (2015.01-2022.12)')
plt.show()

In [ ]:
#조회수 추세 시각화
plt.figure(figsize=(10,5))
dates = pd.date_range(start='2015-01-01', periods=len(df_yt), freq='M')
plt.plot(dates, df_yt['sum_view'])
plt.xlabel('Month')
plt.ylabel('Views')
plt.title('Youtube Views by month (2015.01-2022.12)')
#start = df_yt.index[0]
#end = df_yt.index[-1]
#plt.xlim(start,end)
plt.show()

In [ ]:
#영상 수 추세 시각화
plt.figure(figsize=(10,5))
plt.plot(dates, df_yt['sum_count'])
plt.xlabel('Month')
plt.ylabel('Videos')
plt.title('Number of Youtube Videos by month (2015.01-2022.12)')
plt.show()

In [ ]:
#평균 조회수 시각화
plt.figure(figsize=(10,5))
plt.plot(dates, df_yt['avg_view'])
plt.xlabel('Month')
plt.ylabel('Average views')
plt.title('Average Youtube Views by month (2015.01-2022.12)')
plt.show()

### df_yt_acc: 조회수, 영상 수, 평균조회수, 사고 건수 합친 data frame 만들기

In [ ]:
yt = df_yt.loc[:,['sum_view','sum_count','avg_view']]
acc = df_acc['sum_acc'].to_frame()
df_yt_acc = pd.merge(acc,yt,left_index=True,right_index=True)
df_yt_acc

### 이상치 확인

In [ ]:
# Stack features and targets for joint outlier removal
data = np.column_stack((df_yt_acc['sum_acc'],df_yt_acc['sum_view']))
print(data.shape)

# Calculate IQR
Q1 = np.percentile(data, 25, axis=0)
Q3 = np.percentile(data, 75, axis=0)
IQR = Q3 - Q1

# Define outliers
outlier_mask = np.any((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)), axis=1)

# Filter out the outliers
data_no_outliers = data[~outlier_mask]
print(data_no_outliers.shape)

# Separate features and targets again
acc_no_outliers = data_no_outliers[:, 0]
views_no_outliers = data_no_outliers[:, 1]

In [ ]:
sns.distplot(views_no_outliers) #right skewed => log

In [ ]:
sns.distplot(np.log(views_no_outliers))

In [ ]:
#DF no outliers
log_view_no_outliers = np.log(views_no_outliers)
df_no_outliers = {'log_view': log_view_no_outliers.flatten().tolist(),
        'acc': acc_no_outliers.flatten().tolist()}
df_no_outliers = pd.DataFrame(df_no_outliers)
df_no_outliers

In [ ]:
#Scaling
scaler = MinMaxScaler()
scaled_log_view_no = scaler.fit_transform(np.array(df_no_outliers['log_view']).reshape(-1, 1))
scaled_acc_no = scaler.fit_transform(np.array(df_no_outliers['acc']).reshape(-1, 1))

In [ ]:
#DF Scaled no outliers
df_scaled_no_outliers = {'log_view': scaled_log_view_no.flatten().tolist(),
        'acc': scaled_acc_no.flatten().tolist()}
df_scaled_no_outliers = pd.DataFrame(df_scaled_no_outliers)
df_scaled_no_outliers

### 회귀분석

In [ ]:
model_no_outliers= LinearRegression().fit(scaled_log_view_no, scaled_acc_no)
coef = model_no_outliers.coef_[0]
intercept = model_no_outliers.intercept_
r_squared = model_no_outliers.score(scaled_log_view_no, scaled_acc_no)
print(f"Coefficient: {coef}")
print(f"Intercept: {intercept}")
print(f"R-squared: {r_squared}")

In [ ]:
import scipy.stats as stats

In [ ]:
#상관분석
X = df_scaled_no_outliers['log_view'].values
Y = df_scaled_no_outliers['acc'].values
stats.pearsonr(X,Y)

In [ ]:
pred = model_no_outliers.predict(np.array([0.0, 1.0]).reshape(-1, 1))

plt.figure(figsize=(7, 7))
plt.title("Number of Accident by Youtube Views \nby month (2015.01-2022.12)")
plt.xlabel("MinMax-Scaled log(Youtube Views)")
plt.ylabel("MinMax-Scaled Number of Accident")
plt.scatter(scaled_log_view_no, scaled_acc_no, color='blue',s=10)
plt.plot(scaled_log_view_no,fit.fittedvalues, color="red")
plt.show()

In [ ]:
#회귀계수의 유의성 검정 (t-statistic, p-value)
x = scaled_log_view_no
y = scaled_acc_no
n = len(x)
p = 1  

#t-statistic
mse = np.mean((model_no_outliers.predict(x) - y) ** 2)
se = np.sqrt(mse / (n - p - 1) / np.sum((x - np.mean(x)) ** 2))
t_stat = coef / se

#p-value
p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), n - p - 1))

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

#Check if the coefficient is significant
alpha = 0.05  # Significance level
if p_value < alpha:
    print("The coefficient is statistically significant.")
else:
    print("The coefficient is not statistically significant.")

In [ ]:
#선형 모형 적합
fit = ols('scaled_acc_no ~ scaled_log_view_no', data=df_scaled_no_outliers).fit()
print(fit.summary())

In [ ]:
#정규성 검정
plt.rcParams["figure.figsize"] = (5, 5)
sm.qqplot(fit.resid, fit = True, line = "45", color='grey')
plt.show()

#Shapiro-wilk test 
stats.shapiro(residuals)

In [ ]:
#등분산성 검정
y_pred = model_no_outliers.predict(x)
residuals = y - y_pred

plt.figure(figsize=(6, 6))
plt.scatter(y_pred, residuals, color='blue', s=10)
plt.hlines(y=0, xmin=min(y_pred), xmax=max(y_pred), colors='red', linestyles='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()